In [1]:
import os
import pandas as pd
import numpy as np
from pandas import to_datetime
from datetime import datetime

In [2]:
print("Current Working Directory " , os.getcwd())

Current Working Directory  /Users/aliccia/Desktop/Final Case


In [3]:
os.chdir("/Users/aliccia/Desktop/Final Case")

In [4]:
data = pd.read_csv("subscribers.csv")

In [5]:
data2 = data.dropna(how='any', subset=['attribution_technical','account_creation_date'])

In [6]:
df_dt=to_datetime(data2.account_creation_date)
df1=pd.concat([data2,df_dt],axis=1)

In [7]:
data2["time"]=pd.to_datetime(data2["account_creation_date"])

In [8]:
DATA=data2.set_index("time")

In [9]:
data2['year'] = data2['time'].dt.year
data2['month'] = data2['time'].dt.month
data2['day'] = data2['time'].dt.day

In [10]:
groupby=pd.DataFrame(data2.groupby(["year","month","attribution_technical"])["subid"].count())
groupby.head(20)

subid
year month attribution_technical         
2019 6     affiliate                   78
           bing                         7
           bing_organic                 2
           brand sem intent bing       40
           brand sem intent google    197
           display                      3
           email                      309
           email_blast                109
           facebook                  1318
           facebook_organic            33
           google_organic             131
           influencer                   1
           internal                    32
           organic                    219
           other                       14
           pinterest                   37
           referral                    22
           samsung                      2
           search                     108
           youtube                      1

In [11]:
groupby.to_csv('groupby.csv')

In [12]:
groupby1 = pd.read_csv("groupby.csv")

In [13]:
groupby1.columns

Index(['year', 'month', 'attribution_technical', 'subid'], dtype='object')

In [14]:
groupby2 = groupby1.loc[groupby1['attribution_technical'].isin(['bing','display','facebook','search','youtube'])]
groupby3 = groupby2.rename(columns={'attribution_technical': 'channel', 'subid': 'customer_count'})

In [26]:
groupby3.drop(groupby3.tail(5).index, inplace=True)
groupby3.drop(groupby3.head(5).index, inplace=True)

In [16]:
groupby3.to_csv('groupby3.csv')
groupby3 = pd.read_csv("groupby3.csv")

In [17]:
data = pd.read_csv("channel_spend_graduate.csv")
data.head()

,channel,date,spend_AED
0,bing,201907,400
1,display,201907,12
2,facebook,201907,9000
3,search,201907,13000
4,youtube,201907,90


In [18]:
sub_df = groupby3["customer_count"]
sub_df.head()

0      120
1       44
2    10011
3     1328
4        8
Name: customer_count, dtype: int64

In [19]:
data = data.join(sub_df)
data.head()

,channel,date,spend_AED,customer_count
0,bing,201907,400,120
1,display,201907,12,44
2,facebook,201907,9000,10011
3,search,201907,13000,1328
4,youtube,201907,90,8


In [20]:
bing = data.groupby('channel').get_group('bing')
# sort Brand - ascending order
bing.sort_values(by=['spend_AED'], inplace=True)
bing['margi_cost'] = bing['spend_AED'] - bing['spend_AED'].shift(1)
bing['margi_count'] = bing['customer_count'] - bing['customer_count'].shift(1)
bing['margi_cost'].fillna(300.0, inplace = True)
bing['margi_count'].fillna(28.0, inplace = True)
bing['margi_cac'] = bing.margi_cost / bing.margi_count
bing.drop(["date"], axis = 1, inplace = True) 
print(bing)

   channel  spend_AED  customer_count  margi_cost  margi_count  margi_cac
25    bing        300              28       300.0         28.0  10.714286
0     bing        400             120       100.0         92.0   1.086957
5     bing        900              71       500.0        -49.0 -10.204082
10    bing       1000              84       100.0         13.0   7.692308
15    bing       1100              86       100.0          2.0  50.000000
20    bing       1300             103       200.0         17.0  11.764706
30    bing       2100             172       800.0         69.0  11.594203
35    bing       3700             303      1600.0        131.0  12.213740


<ipython-input-20-b908b1e783c7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bing.sort_values(by=['spend_AED'], inplace=True)
<ipython-input-20-b908b1e783c7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bing['margi_cost'] = bing['spend_AED'] - bing['spend_AED'].shift(1)
<ipython-input-20-b908b1e783c7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [27]:
display = data.groupby('channel').get_group('display')
display.sort_values(by=['spend_AED'], inplace=True)
display['margi_cost'] = display['spend_AED'] - display['spend_AED'].shift(1)
display['margi_count'] = display['customer_count'] - display['customer_count'].shift(1)
display['margi_cost'].fillna(12.0, inplace = True)
display['margi_count'].fillna(44.0, inplace = True)
display['margi_cac'] = display.margi_cost / display.margi_count
display.drop(["date"], axis = 1, inplace = True) 
print(display)

    channel  spend_AED  customer_count  margi_cost  margi_count  margi_cac
1   display         12              44        12.0         44.0   0.272727
11  display         13              12         1.0        -32.0  -0.031250
16  display         19              50         6.0         38.0   0.157895
6   display         20              74         1.0         24.0   0.041667
21  display         29              90         9.0         16.0   0.562500
26  display         31             112         2.0         22.0   0.090909
36  display         94             341        63.0        229.0   0.275109
31  display        148             536        54.0        195.0   0.276923


<ipython-input-27-fba072d8c447>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  display.sort_values(by=['spend_AED'], inplace=True)
<ipython-input-27-fba072d8c447>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  display['margi_cost'] = display['spend_AED'] - display['spend_AED'].shift(1)
<ipython-input-27-fba072d8c447>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [28]:
facebook = data.groupby('channel').get_group('facebook')
facebook.sort_values(by=['spend_AED'], inplace=True)
facebook['margi_cost'] = facebook['spend_AED'] - facebook['spend_AED'].shift(1)
facebook['margi_count'] = facebook['customer_count'] - facebook['customer_count'].shift(1)
facebook['margi_cost'].fillna(9000.0, inplace = True)
facebook['margi_count'].fillna(10011.0, inplace = True)
facebook['margi_cac'] = facebook.margi_cost / facebook.margi_count
facebook.drop(["date"], axis = 1, inplace = True) 
print(facebook)

     channel  spend_AED  customer_count  margi_cost  margi_count  margi_cac
2   facebook       9000           10011      9000.0      10011.0   0.899011
12  facebook      10500            6129      1500.0      -3882.0  -0.386399
32  facebook      11000            6289       500.0        160.0   3.125000
7   facebook      13000            9855      2000.0       3566.0   0.560852
37  facebook      14000            7988      1000.0      -1867.0  -0.535619
27  facebook      16000            9288      2000.0       1300.0   1.538462
17  facebook      17000            9489      1000.0        201.0   4.975124
22  facebook      23000           12995      6000.0       3506.0   1.711352


<ipython-input-28-0ef1ed125b9c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facebook.sort_values(by=['spend_AED'], inplace=True)
<ipython-input-28-0ef1ed125b9c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facebook['margi_cost'] = facebook['spend_AED'] - facebook['spend_AED'].shift(1)
<ipython-input-28-0ef1ed125b9c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [29]:
search = data.groupby('channel').get_group('search')
search.sort_values(by=['spend_AED'], inplace=True)
search['margi_cost'] = search['spend_AED'] - search['spend_AED'].shift(1)
search['margi_count'] = search['customer_count'] - search['customer_count'].shift(1)
search['margi_cost'].fillna(13000.0, inplace = True)
search['margi_count'].fillna(1328.0, inplace = True)
search['margi_cac'] = search.margi_cost / search.margi_count
search.drop(["date"], axis = 1, inplace = True) 
print(search)

   channel  spend_AED  customer_count  margi_cost  margi_count  margi_cac
3   search      13000            1328     13000.0       1328.0   9.789157
8   search      18500            1939      5500.0        611.0   9.001637
13  search      19000            1852       500.0        -87.0  -5.747126
18  search      24000            2333      5000.0        481.0  10.395010
23  search      25000            2510      1000.0        177.0   5.649718
28  search      38000            3770     13000.0       1260.0  10.317460
33  search      41000            4073      3000.0        303.0   9.900990
38  search      44000            4300      3000.0        227.0  13.215859


<ipython-input-29-30bf0e0b73d8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search.sort_values(by=['spend_AED'], inplace=True)
<ipython-input-29-30bf0e0b73d8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search['margi_cost'] = search['spend_AED'] - search['spend_AED'].shift(1)
<ipython-input-29-30bf0e0b73d8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [30]:
youtube = data.groupby('channel').get_group('youtube')
youtube.sort_values(by=['spend_AED'], inplace=True)
youtube['margi_cost'] = youtube['spend_AED'] - youtube['spend_AED'].shift(1)
youtube['margi_count'] = youtube['customer_count'] - youtube['customer_count'].shift(1)
youtube['margi_cost'].fillna(90.0, inplace = True)
youtube['margi_count'].fillna(8.0, inplace = True)
youtube['margi_cac'] = youtube.margi_cost / youtube.margi_count
youtube.drop(["date"], axis = 1, inplace = True) 
print(youtube)

    channel  spend_AED  customer_count  margi_cost  margi_count  margi_cac
4   youtube         90               8        90.0          8.0  11.250000
14  youtube        100               2        10.0         -6.0  -1.666667
19  youtube        130              11        30.0          9.0   3.333333
9   youtube        180              15        50.0          4.0  12.500000
24  youtube        550              47       370.0         32.0  11.562500
29  youtube        900              76       350.0         29.0  12.068966
39  youtube       2420             205      1520.0        129.0  11.782946
34  youtube       4360             370      1940.0        165.0  11.757576


<ipython-input-30-e0233426f9d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youtube.sort_values(by=['spend_AED'], inplace=True)
<ipython-input-30-e0233426f9d7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youtube['margi_cost'] = youtube['spend_AED'] - youtube['spend_AED'].shift(1)
<ipython-input-30-e0233426f9d7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/